In [1]:
import imaplib
import re

In [2]:
FROM_EMAIL  = "max.y.leander@gmail.com"
FROM_PWD    = "JjnpStrossle123"
SMTP_SERVER = "imap.gmail.com"
SMTP_PORT   = 993

In [3]:
mail = imaplib.IMAP4_SSL(SMTP_SERVER)
mail.login(FROM_EMAIL, FROM_PWD)

('OK', [b'max.y.leander@gmail.com authenticated (Success)'])

In [4]:
list_response_pattern = re.compile(r'\((?P<flags>.*?)\) "(?P<delimiter>.*)" (?P<name>.*)')
def parse_list_response(line):
    flags, delimiter, mailbox_name = list_response_pattern.match(line).groups()
    mailbox_name = mailbox_name.strip('"')
    return (flags, delimiter, mailbox_name)

In [5]:
_, box_list = mail.list()
for line in box_list:
    _, _, mailbox_name = parse_list_response(line.decode('utf-8'))
    print(mailbox_name)
    mail.select(f'"{mailbox_name}"')
    try:
        _, data = mail.search(None, 'ALL')
        ids = data[0].split()
        print(len(ids))
    except Exception:
        print(f'Could not search for e-mails in "{mailbox_name}"')

BLN
178
Claudia
85
INBOX
1383
Personal
1
Receipts
0
Skickad e-post
0
Travel
0
Work
0
[Gmail]
Could not search for e-mails in "[Gmail]"
[Gmail]/All Mail
37054
[Gmail]/Drafts
0
[Gmail]/Important
21099
[Gmail]/Sent Mail
4486
[Gmail]/Spam
11
[Gmail]/Starred
2
[Gmail]/Trash
0


In [6]:
mail.select(f'"[Gmail]/All Mail"')
mail_ids = set()
to_adresses = ['cederberg.erik@gmail.com', 
               'erik.cederberg@kontigo.se',
               'johannes.siven@gmail.com', 
               'johannessiven@gmail.com',
               'steen.eriksteen@gmail.com', 
               'valdemar@scandikitchen.co.uk',
               'valdemar.lundh@gmail.com'
              ]
for adr1 in to_adresses[:-1]:
    for adr2 in to_adresses[1:]:
        if adr1 == adr2:
            continue
        _, data = mail.search(None, f'(TO "{adr1}" TO "{adr2}")')
        print(f'{adr1} + {adr2} = {len(mail_ids)}')
        mail_ids.update(data[0].split())
mail_ids = list(mail_ids)
print(len(mail_ids))

cederberg.erik@gmail.com + erik.cederberg@kontigo.se = 0
cederberg.erik@gmail.com + johannes.siven@gmail.com = 1
cederberg.erik@gmail.com + johannessiven@gmail.com = 1230
cederberg.erik@gmail.com + steen.eriksteen@gmail.com = 1299
cederberg.erik@gmail.com + valdemar@scandikitchen.co.uk = 1876
cederberg.erik@gmail.com + valdemar.lundh@gmail.com = 1888
erik.cederberg@kontigo.se + johannes.siven@gmail.com = 1905
erik.cederberg@kontigo.se + johannessiven@gmail.com = 1905
erik.cederberg@kontigo.se + steen.eriksteen@gmail.com = 1905
erik.cederberg@kontigo.se + valdemar@scandikitchen.co.uk = 1911
erik.cederberg@kontigo.se + valdemar.lundh@gmail.com = 1911
johannes.siven@gmail.com + erik.cederberg@kontigo.se = 1911
johannes.siven@gmail.com + johannessiven@gmail.com = 1911
johannes.siven@gmail.com + steen.eriksteen@gmail.com = 1980
johannes.siven@gmail.com + valdemar@scandikitchen.co.uk = 2807
johannes.siven@gmail.com + valdemar.lundh@gmail.com = 2905
johannessiven@gmail.com + erik.cederberg@ko

In [7]:
from html.parser import HTMLParser

class MLStripper(HTMLParser):
    def __init__(self):
        self.reset()
        self.strict = False
        self.convert_charrefs= True
        self.fed = []
    def handle_data(self, d):
        self.fed.append(d)
    def get_data(self):
        return ''.join(self.fed)

def strip_tags(html):
    s = MLStripper()
    s.feed(html)
    return s.get_data()

In [26]:
def parse_content(msg):
    content = []
    if msg.is_multipart():
        for payload in msg.get_payload():
            content.append(payload.get_payload(decode=True).decode('utf-8'))
    else:
        content.append(msg.get_payload(decode=True).decode('utf-8'))
    return strip_tags('\n'.join(content))

In [28]:
import email
_, data = mail.fetch(mail_ids[0], '(RFC822)')
msg = email.message_from_string(data[0][1].decode('utf-8'))
parsed_msg = parse_content(msg)

In [36]:
class MsgNode:
    def __init__(self):
        self.msg = []
        self.parent = None

In [37]:
current_msg = MsgNode()
level = 0
for line in parsed_msg.split('\n'):
    if line.startswith('>' * level):
        current_msg.msg.append(line)
    elif line.startswith('>' * (level + 1)):
        next_msg = MsgNode()
        current_msg.parent = next_msg
        current_msg = next_msg
        current_msg.msg.append(line)